In [7]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os



In [8]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data_re_id.npy');
path_to_labels = path.join(root_folder,'data/train_labels_re_id.npy');
#path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 75;
logging_interval = 30
dataset_divider = 0.2

In [10]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
nb_of_classes = torch_labels.unique().size()[0]

In [11]:
print(nb_of_classes)

499


In [12]:
net = cnn.ft_net( nb_of_classes, 0);
net.to(device)

ft_net(
  (model): ResNet(
    (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU

In [13]:
if (len(torch_labels.size()) == 2):
    torch_labels = torch.squeeze(torch_labels)

In [ ]:
# vertical flip
train_data_original, test_data_original = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)
train_data_flipped, test_data_flippedl = cnn.GenerateTrainAndTestDataset(torch_data * -1, torch_labels, 0.2)



In [ ]:
train_data = torch.utils.data.ConcatDataset((train_data_original, train_data_flipped));
test_data = torch.utils.data.ConcatDataset((test_data_original, test_data_flippedl));


In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)





In [ ]:
print(torch_data.size())
print(torch_labels.size())

In [ ]:
print(torch_data[0,:])

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
exp_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 25, 45, 65], gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/re_id/resnet18_num_classes_' + str(nb_of_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, train_loader)
  cnn.Test(net, device, criterion, test_loader)

In [ ]:
cnn.Test(net, device, criterion, test_loader)